# Using Action Tools


**Step 1**: Install NuGet packages

To get started with Azure AI Foundry, you need to install the required NuGet packages:

In [61]:
#r "nuget: Microsoft.SemanticKernel"
#r "nuget: Microsoft.Extensions.AI.AzureAIInference, 9.5.0-preview.1.25265.7"
#r "nuget: Microsoft.SemanticKernel.Agents.Core"
#r "nuget: DotNetEnv, 3.1.0"
#r "nuget: Microsoft.SemanticKernel.Agents.AzureAI, 1.56.0-preview"

Installed Packages DotNetEnv, 3.1.0 Microsoft.Extensions.AI.AzureAIInference, 9.5.0-preview.1.25265.7 Microsoft.SemanticKernel, 1.56.0 Microsoft.SemanticKernel.Agents.AzureAI, 1.56.0-preview Microsoft.SemanticKernel.Agents.Core, 1.56.0

**Step 2**: Read environment variables

In this step, we load these variables from a `.env` file (if present) so that they can be accessed by the application.

In [62]:
using DotNetEnv;
using System.IO;

var envFilePath = Path.Combine(Environment.CurrentDirectory, "../..", ".env");
if (File.Exists(envFilePath))
{
    Env.Load(envFilePath);
    Console.WriteLine($"Loaded environment variables from {envFilePath}");
}
else
{
    Console.WriteLine($"No .env file found at {envFilePath}");
}

Loaded environment variables from d:\personal\aiagent-workshop\notebooks\ai-foundry\../..\.env


**Step 3**: Instantiate the Project Client

Create a `PersistentAgentsClient` using your Azure AI Foundry project endpoint.

Note that **DefaultAzureCredential** is used for authentication. Run `az login --use-device-code` if you haven't already.

In [63]:
using Microsoft.SemanticKernel;
using Microsoft.SemanticKernel.ChatCompletion; 
using Microsoft.SemanticKernel.Agents.AzureAI;
using Azure.Identity;

var projectEndpoint = Environment.GetEnvironmentVariable("AZURE_AI_PROJECT_ENDPOINT");
#pragma warning disable SKEXP0110 // Type is for evaluation purposes only and is subject to change or removal in future updates. Suppress this diagnostic to proceed.
var client = AzureAIAgent.CreateAgentsClient(projectEndpoint, new DefaultAzureCredential());
#pragma warning restore SKEXP0110 // Type is for evaluation purposes only and is subject to change or removal in future updates.

# Bike Inventory Agent
This section demonstrates how to create a bike order management agent that can handle customer orders, and provide an update on the status of an order.

**Step 1**: Define your agent

In this step, define the attributes for the agent, including its name, instructions, tools (if any), and the model to use.

In [ ]:
using Microsoft.SemanticKernel;
using Microsoft.SemanticKernel.ChatCompletion;
using Microsoft.SemanticKernel.Agents.AzureAI;
using Azure.Identity;
using Azure.AI.Agents.Persistent;


var modelId = Environment.GetEnvironmentVariable("MODEL");
var agentName = "ContosoInventoryAgent";
var agentInstructions = @"
You are a knowledgeable and friendly assistant for the Contoso Bike Store.
Your job is to help customers find information about bikes, check inventory using the Contoso Bike Store API.
Always provide clear, concise, and accurate responses. If you need more details from the user, ask clarifying questions.
Use the API to retrieve up-to-date information.
"; 

var filePath = Path.Combine(Environment.CurrentDirectory, "../../resources/bike-inventory-api-swagger.json");
var fileData = File.ReadAllBytes(filePath);

OpenApiAnonymousAuthDetails oaiAuth = new();
OpenApiToolDefinition openApiTool = new(
    name: "ContosoBikeStoreAPI",
    description: "The API provides information about the bikes in the catalog for the Contoso Bike Store.",
    spec: BinaryData.FromBytes(fileData),
    openApiAuthentication: oaiAuth,
    defaultParams: ["format"]
);

var agentDefinition = await client.Administration.CreateAgentAsync(
    modelId,
    name: agentName,
    instructions: agentInstructions,
    tools: [openApiTool],
    toolResources: null);

**Step 2**: Create the agent

Finally, we create the agent using the `CreateAgentAsync` method of the `PersistentAgentsClient`. This method takes the agent definition and returns the created agent.

In [70]:
using Microsoft.SemanticKernel;
using Microsoft.SemanticKernel.ChatCompletion; 
using Microsoft.SemanticKernel.Agents.AzureAI;

#pragma warning disable SKEXP0110 // Type is for evaluation purposes only and is subject to change or removal in future updates. Suppress this diagnostic to proceed.
       
var agent = new AzureAIAgent(agentDefinition, client);

#pragma warning restore SKEXP0110 // Type is for evaluation purposes only and is subject to change or removal in future updates.

**Step 3**: Create a new thread for the agent

In [71]:
using Microsoft.SemanticKernel;
using Microsoft.SemanticKernel.ChatCompletion; 
using Microsoft.SemanticKernel.Agents.AzureAI;

#pragma warning disable SKEXP0110 // Type is for evaluation purposes only and is subject to change or removal in future updates. Suppress this diagnostic to proceed.
       
var thread = new AzureAIAgentThread(client);

#pragma warning restore SKEXP0110 // Type is for evaluation purposes only and is subject to change or removal in future updates.

**Step 4** : Interact with the Agent

In [72]:
#pragma warning disable SKEXP0110 // Type is for evaluation purposes only and is subject to change or removal in future updates. Suppress this diagnostic to proceed.

    var invokeAgentAsync = async (AzureAIAgentThread agentThread, string input) =>
    {
        ChatMessageContent message = new(AuthorRole.User, input);
        await foreach (ChatMessageContent response in agent.InvokeAsync(message, agentThread))
        {
            Console.WriteLine($"{response.Role}: {response.Content}");
        }
    };

    #pragma warning disable SKEXP0110 // Type is for evaluation purposes only and is subject to change or removal in future updates. Suppress this diagnostic to proceed.

    var input = "What bikes do you have in stock?";
    await invokeAgentAsync(thread, input);
    

assistant: Here is an overview of the bikes currently in stock at the Contoso Bike Store:

1. **Contoso Roadster 200** - Lightweight road bike for speed and endurance, price: $899.99 (Stock: 12, Seattle - Downtown).
2. **Contoso Mountain X1** - Durable mountain bike for tough terrains, price: $1,199.99 (Stock: 8, Seattle - Northgate).
3. **Contoso City Cruiser** - Comfortable bike for daily city commuting, price: $499.99 (Stock: 15, Portland - Central).
4. **Contoso Kids Sprint** - Safe and colorful bike for children, price: $299.99 (Stock: 20, Portland - Eastside).
5. **Contoso E-Bike Pro** - Electric bike with long battery life, price: $1,799.99 (Stock: 5, San Francisco - Market Street).
6. **Contoso Gravel 360** - Versatile gravel bike for mixed surfaces, price: $1,099.99 (Stock: 10, San Francisco - Mission).
7. **Contoso Folding Go** - Compact folding bike for easy storage, price: $649.99 (Stock: 7, Austin - Downtown).
8. **Contoso Tandem Duo** - Tandem bike for two riders, price: 

**Step 5**: Clean up Resources

Delete the agent and the thread to clean up resources.

In [73]:
   await thread.DeleteAsync();
   await client.Administration.DeleteAgentAsync(agent.Id);